### Imports

In [1]:
import os
import imageio.v3 as iio
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import cv2

In [2]:
red = cv2.resize(cv2.imread(r'imSeg/red.jpg'),[1200, 675])
blue = cv2.resize(cv2.imread(r'imSeg/blue.jpg'),[1200, 675])
sarena = cv2.imread(r'imSeg/sarena.jpg')
male = cv2.imread(r'imSeg/male.jpg')
gcloth = cv2.imread(r'imSeg/gcloth.jpg')
hard = cv2.imread(r'imSeg/hard.jpg')
test = cv2.imread(r'imSeg/test_trouble.jpg')

## Coler conversion and thresholding

In [3]:
def get_hsv_values_from_pixel(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        hsv_image = param['hsv_image']
        pixel_hsv = hsv_image[y, x]
        print(f"HSV value at ({x}, {y}): {pixel_hsv}")

In [4]:
def tune_hsv_range(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    cv2.imshow("get HSV", image)
    cv2.setMouseCallback("get HSV", get_hsv_values_from_pixel, {'hsv_image': hsv_image})
    cv2.waitKey(2000)
    cv2.destroyAllWindows()

In [5]:
def color_segment_image(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    lower_green = np.array([28, 30, 50])
    upper_green = np.array([42, 256, 256])

    mask = cv2.inRange(hsv_image, lower_green, upper_green)

    segmented_image = cv2.bitwise_and(image, image, mask=mask)
    
    return segmented_image

### good image values 
(lower_green = np.array([30, 60, 130])
upper_green = np.array([40, 256, 256]))

#Good for jannik side:
lower_green = np.array([28, 18, 130])
    upper_green = np.array([89, 256, 256])

    with background: lower_green = np.array([28, 30, 50])
    upper_green = np.array([42, 256, 256])
    

## Blurring and canny edge detection

In [6]:
def canny_edge_detect(segmented_image):
    gray_segmented = cv2.cvtColor(segmented_image, cv2.COLOR_BGR2GRAY)

    blurred_gray = cv2.GaussianBlur(gray_segmented, (11, 11), 0)

    threshold1 = 50
    threshold2 = 150

    edges = cv2.Canny(blurred_gray, threshold1, threshold2, apertureSize=3)

    return edges

In [63]:
test = cv2.imread(r'reportMedia\steadyKNN.png')

edged = canny_edge_detect(color_segment_image(test))

cv2.imshow("get HSV", edged)
cv2.waitKey(2000)
cv2.destroyAllWindows()

## Hough transform

In [ ]:
def detect_tennis_ball_hough(canny_edges_image, original_image):

    # Make a copy of the original image to draw on
    output_image = original_image.copy()

    # quick scaling (dirty but makes detection less resolution dependent)
    img_height = output_image.shape[0]

    scaling = img_height * 0.001

    circles = cv2.HoughCircles(
        canny_edges_image,
        cv2.HOUGH_GRADIENT,
        dp=3,          # Inverse ratio of accumulator resolution (1 for same resolution, 2 for half)
        param1=100,      # Upper threshold for the internal Canny edge detector (e.g., 50-200)
        param2= 20 * scaling,       # Accumulator threshold for the circle centers. Lower -> more false circles. (e.g., 20-100)
        minRadius=3, # Minimum radius of the circles to detect
        maxRadius=15,  # Maximum radius of the circles to detect
        minDist= 75, # Minimum distance between centers (e.g., about the ball's diameter)
    )

    

    detected_ball_info = None
    if circles is not None:
        circles = np.uint16(np.around(circles))
        # Take the first detected circle for simplicity (you'd add data association here)
        i = circles[0, 0]
        center_x, center_y, radius = i[0], i[1], i[2]
        #cv2.circle(output_image, (center_x, center_y), radius, (0, 0, 200), 2)
        #cv2.circle(output_image, (center_x, center_y), 2, (200, 80, 200), 3)
        detected_ball_info = (center_x, center_y) # Only returning (x,y) for Kalman measurement
    return output_image, detected_ball_info

In [8]:
image = test

resize = cv2.resize(image,[1200, 675])

segmented_image = color_segment_image(test)

#cannyed_image = canny_edge_detect(segmented_image)


#output_image, circles = detect_tennis_ball_hough(cannyed_image, image, 7, 50)

#print(circles)

#tune_hsv_range(image)



cv2.imshow("Original Image - Click to get HSV", segmented_image)
cv2.waitKey(2000)
cv2.destroyAllWindows()

tune_hsv_range(test)



HSV value at (730, 144): [33 98 81]


## Kalman filter

In [69]:
class KalmanFilterTracker:
    def __init__(self):
        # Initialize Kalman Filter
        # 4 state variables: [x, y, vx, vy]
        # 2 measurement variables: [x_measured, y_measured]
        self.kf = cv2.KalmanFilter(4, 2)

        # Transition Matrix (A) - Constant velocity model
        self.kf.transitionMatrix = np.array([[1, 0, 1, 0],
                                             [0, 1, 0, 1],
                                             [0, 0, 1, 0],
                                             [0, 0, 0, 1]], dtype=np.float32)

        # Measurement Matrix (H) - We measure position directly
        self.kf.measurementMatrix = np.array([[1, 0, 0, 0],
                                             [0, 1, 0, 0]], dtype=np.float32)

        # Process Noise Covariance (Q) - Tune these!
        # Accounts for uncertainty in our model (e.g., ball acceleration)
        self.kf.processNoiseCov = np.eye(4, dtype=np.float32) * 0.1 # Example: uniform small noise
        # More refined:
        # self.kf.processNoiseCov = np.array([[1, 0, 0, 0],   # Noise for x
        #                                     [0, 1, 0, 0],   # Noise for y
        #                                     [0, 0, 0.05, 0], # Noise for vx
        #                                     [0, 0, 0, 0.05]], dtype=np.float32)

        # Measurement Noise Covariance (R) - Tune these!
        # Accounts for noise in our detection (Hough output jitter)
        self.kf.measurementNoiseCov = np.eye(2, dtype=np.float32) * 50 # Example: uniform measurement noise
        # More refined:
        # self.kf.measurementNoiseCov = np.array([[50, 0],
        #                                         [0, 50]], dtype=np.float32)

        # Error Covariance (P) - Initial uncertainty in state. Large values are typical.
        self.kf.errorCovPost = np.eye(4, dtype=np.float32) * 1000

        # Initial state (x,y,vx,vy) - Will be set after first detection
        self.kf.statePost = np.zeros((4, 1), dtype=np.float32)

        self.is_initialized = False

    def predict(self):
        """
        Predicts the next state of the tracked object.
        Returns: Tuple (predicted_x, predicted_y)
        """
        predicted_state = self.kf.predict()
        predicted_x = int(predicted_state[0])
        predicted_y = int(predicted_state[1])
        return (predicted_x, predicted_y)

    def update(self, measurement):
        """
        Updates the Kalman filter with a new measurement.
        Args:
            measurement (tuple): (x, y) coordinates of the detected object.
        Returns: Tuple (estimated_x, estimated_y)
        """
        if not self.is_initialized:
            # Initialize state with the first measurement
            self.kf.statePost = np.array([[measurement[0]],
                                          [measurement[1]],
                                          [0.], # Initial vx
                                          [0.]], dtype=np.float32) # Initial vy
            self.is_initialized = True
            # For the very first update, predict and correct are essentially the same as init.
            # We can return the measurement itself or run a quick predict/correct.
            # For simplicity, let's just use the measurement as the first estimated state.
            return measurement[0], measurement[1]
        else:
            # Create the measurement vector for Kalman filter
            np_measurement = np.array([[measurement[0]],
                                       [measurement[1]]], dtype=np.float32)

            # Correct the state based on the measurement
            estimated_state = self.kf.correct(np_measurement)
            estimated_x = int(estimated_state[0])
            estimated_y = int(estimated_state[1])
            return (estimated_x, estimated_y)

    def get_current_state(self):
        """
        Returns the current estimated state (x, y, vx, vy).
        """
        return self.kf.statePost.flatten().tolist()

## Video reading

In [78]:
def process_video_file(video_path):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return

    fgbg = cv2.createBackgroundSubtractorKNN(history=8, dist2Threshold=3000, detectShadows=False)

    MOGbg = cv2.createBackgroundSubtractorMOG2(history=8, varThreshold=300, detectShadows=False)


    # Loop through frames
    while True:
        ret, frame = cap.read()
        frame = cv2.resize(frame,[854, 480])

        if not ret:
            print("End of video stream or error reading frame.")
            break

        kalman_tracker = KalmanFilterTracker()

        foreground_mask = fgbg.apply(frame)
        #MOG_foreground = MOGbg.apply(frame)

        segmented_foreground = cv2.bitwise_and(frame, frame, mask=foreground_mask)
        #segmented_MOG_foreground = cv2.bitwise_and(frame, frame, mask=MOG_foreground)

        color_segemented = color_segment_image(segmented_foreground)

        

        cannyed_image = canny_edge_detect(color_segemented)
        inpainted_hough, detected_ball_pos = detect_tennis_ball_hough(cannyed_image, frame)

        # --- Kalman Filter Integration ---
        predicted_point = kalman_tracker.predict() # Always predict
        cv2.circle(inpainted_hough, predicted_point, 5, (0, 0, 255), -1) # Red for prediction
        cv2.putText(inpainted_hough, "Predicted", (predicted_point[0] + 10, predicted_point[1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

        if detected_ball_pos:
            # If a detection is found, update the Kalman filter
            estimated_x, estimated_y = kalman_tracker.update(detected_ball_pos)
            # Draw corrected point (blue circle)
            cv2.circle(inpainted_hough, (estimated_x, estimated_y), 5, (255, 0, 0), -1)
            cv2.putText(inpainted_hough, "Estimated", (estimated_x + 10, estimated_y + 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
        else:
            # If no detection, the filter relies purely on its prediction
            print(f"No detection in frame {cap.get(cv2.CAP_PROP_POS_FRAMES)}, relying on prediction.")


        cv2.imshow('Video Feed (Original)', segmented_foreground)
        cv2.imshow('Video Feed (Grayscale)', frame)
        cv2.imshow('Video Feed (frame)', inpainted_hough) 
        #print(circles)

        # increase waitkey value for fewer frames per second
        if cv2.waitKey(100) & 0xFF == ord('q'):
            print("Exiting video display.")
            break

    cap.release()
    cv2.destroyAllWindows()

In [77]:
Jannik_side_view_480p_trim = cv2.VideoCapture(r'vidSeg/Jannik_side_view_480p_trim.mp4')
Back_view_720p_trim = cv2.VideoCapture(r'vidSeg/Back_view_720p_trim.mp4')

process_video_file(r'vidSeg/close_trim.mp4')

C:\Users\lizzy\AppData\Local\Temp\ipykernel_13940\2316264213.py:48: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted_x = int(predicted_state[0])
C:\Users\lizzy\AppData\Local\Temp\ipykernel_13940\2316264213.py:49: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted_y = int(predicted_state[1])


Exiting video display.
